# Million-scale squares on multigraphs with 🍇 GRAPE 🍇
In this tutorial, I will show you how to use the [GRAPE library](https://github.com/AnacletoLAB/grape) to count the number of squares in multigraphs. We will first compute a vertex cover of the graph using GRAPE, [as extensively covered in this previous tutorial](https://github.com/AnacletoLAB/grape/blob/main/tutorials/Billion-scale%202-approximated%20vertex%20cover%20with%20GRAPE.ipynb), and then use this vertex cover to efficiently count the squares in the graph by using our new version of [this awesome algorithm by Oded Green and David Bader](https://davidbader.net/publication/2013-g-ba/2013-g-ba.pdf). This version is faster and handles both graphs with self-loops and multigraphs.

Find the source code for the [global version here](https://github.com/AnacletoLAB/ensmallen/blob/96fd96a9888d4ac8cff44d25e07c49658c5f86e1/graph/src/polygons.rs#L162) and the [per-node version here](https://github.com/AnacletoLAB/ensmallen/blob/96fd96a9888d4ac8cff44d25e07c49658c5f86e1/graph/src/polygons.rs#L319).

The key difference between the algorithms for global square counts and square counts per node is the use of atomic instructions, specifically [fetch add](https://en.wikipedia.org/wiki/Fetch-and-add).

I will explain the concept of a squares and its importance in square counting, and what square counting is for. We will touch upon on basic graph concepts such as self-loops and multigraphs, multisets and multiplicity functions. By the end of the tutorial, you will have a good understanding of how to use [GRAPE](https://github.com/AnacletoLAB/grape) to count the squares in a graph and apply this knowledge to your projects.

[Remember to ⭐ GRAPE!](https://github.com/AnacletoLAB/grape)

### What is GRAPE?
[🍇🍇 GRAPE 🍇🍇](https://github.com/AnacletoLAB/grape) is a graph processing and embedding library that enables users to easily manipulate and analyze graphs. With [GRAPE](https://github.com/AnacletoLAB/grape), users can efficiently load and preprocess graphs, generate random walks, and apply various node and edge embedding models. Additionally, [GRAPE](https://github.com/AnacletoLAB/grape) provides a fair and reproducible evaluation pipeline for comparing different graph embedding and graph-based prediction methods.

![features in GRAPE](https://github.com/AnacletoLAB/grape/raw/main/images/sequence_diagram.png?raw=true)

## Squares in graphs
In graph theory, **a square is a simple cycle of four vertices**. A square is also known as a 4-cycle.

A square can be represented by four vertices and the four edges connecting them. For example, in the following graph:

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/squares.png?raw=true" width=200 />

There is one quare, formed by vertices `1`, `2`, `3` and `4`. The square is represented by the four edges connecting these vertices: `(1,2)`, `(2,3)`, `(3, 4)`, and `(4,1)`.

### How many squares are in this graph?
In this graph we have one square, but also each of the nodes has one square. So, if we sum all of the squares of the nodes in the graph, we would get three squares.

### Why should you care about squares of each node?
[Squares](https://mathworld.wolfram.com/SquareGraph.html), analogously to triangles, are an important concept in graph theory because they represent a basic unit of connectivity in a graph. Knowing exactly how connected each node is, and more importantly, the various areas of the graphs, is an extremely important analytics tool: it allows to plan and test out different approaches on the different areas of the graph, which may have massively different densities. Areas with high density, i.e. connectivity, may currespond to an area where one could execute model predictions easily as we have a lot of information. Conversely, areas with very low density may be areas where we should make an effort to find more knowledge. It is likely that a model that performs well in areas with low density may not be the same model that performs well in high density areas.

We discuss an analogous [degree-based Goldilock holdout in this previous tutorial](https://github.com/AnacletoLAB/grape/blob/main/tutorials/Graph_holdouts_using_GRAPE.ipynb).

Squares also have several applications in various fields, including social network analysis, machine learning, and data mining.

### What is square counting?
The square count problem is the problem of counting the number of squares in a graph. It is a subproblem of more general cycle counting problems, such as counting the number of cycles of a given length in a graph.

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/squares_counting.png?raw=true" width=500 />

#### Why should I count squares?
The square count problem has several applications in various fields, including social network analysis, machine learning, and data mining. In these fields, the number of squares in a graph is often used as a measure of the graph's structure and connectivity. In machine learning and data mining, the square count problem can be used to identify patterns and trends in large data sets, and can be used for tasks such as general graphs node embedding, i.e. not specific to a single graph.

### Some topological quirks we need to look out for 
In several real-world graphs there exist topological peculiarities than can be forgotten while designing algorithms for graphs. The two that are relevant for square counting are self-loops and multigraphs, i.e. graphs characterized by nodes connected by multiple edges.

#### Multigraphs
In a multi-graph nodes may be connected by multiple edges. These graphs can be, for instance, knowledge graphs where various nodes are connected by multiple edges representing different types of relationships.

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/multigraph.png?raw=true" width=400 />

##### Multisets
While we are often used to think of node neighbours as sets, for multi-graphs they may take the shape of [multisets](https://en.wikipedia.org/wiki/Multiset#:~:text=In%20mathematics%2C%20a%20multiset%20(or,that%20element%20in%20the%20multiset.), i.e. sets where one or more elements appear multiple times.

We use the notation $m_{\mathcal{N}(v)}(w): V \to \mathbb{N}$ to define the **multiplicity function** that returns for a given node $w \in V$, how many times it appears in the immediate neighbourhood of node $v \in V$, represented as $\mathcal{\mathcal{N}(v)}$. In the python pseudocode, we represent the function $m_{\mathcal{N}(v)}(w)$ as `multiplicity(neighbours(v), w)`.

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/set_and_multiset.png?raw=true" width=400 />

#### Self-loops
The name is rather self-explanatory: self-loops are edges that start and end in the same node. In multi-graphs, nodes may have several self-loops.

<img src="https://github.com/AnacletoLAB/grape/blob/main/images/multigraph_with_self_loops.png?raw=true" width=400 />

## Installing GRAPE
First, we install the GRAPE library from PyPI:

In [4]:
!pip install grape -qU


[notice] A new release of pip available: 22.3.1 -> 23.0.1
[notice] To update, run: pip install --upgrade pip


## Experiments
Welcome to the experiments section of this tutorial! In this section, we will put our knowledge into practice by applying the squares counting algorithm on several graphs, including [a few protein-protein interaction graphs from STRING PPI](https://string-db.org/), and then we are going to scale up our targer goals to the [KGCOVID19 knowledge graph](https://www.cell.com/patterns/fulltext/S2666-3899(20)30203-8?_returnURL=https%3A%2F%2Flinkinghub.elsevier.com%2Fretrieve%2Fpii%2FS2666389920302038%3Fshowall%3Dtrue), and the [Friendster graph](https://networkrepository.com/friendster.php).

We run these experiments on a machine with 24 threads and 12 cores.

In my machine I have 12 cores and 24 threads. You can estimate the expected computation time by interpolating the time estimates on 24 threads and the amount you have:

In [5]:
import os

os.cpu_count()

24

Also, this machine has about `128GB` of RAM:

In [6]:
import psutil
    
psutil.virtual_memory().total / 1024**3 # total physical memory in Bytes

125.70906066894531

### Evaluating the impact of the vertex cover heuristic
One may employ several different strategies to build a vertex cover. Here, we consider six strategies overall, with three node ordering approaches and two addition schemas: either only the source node, or both the source and destination nodes. [Please do check out this tutorial to learn more about the vertex cover algorithm we employ here.](https://github.com/AnacletoLAB/grape/blob/main/tutorials/Billion-scale%202-approximated%20vertex%20cover%20with%20GRAPE.ipynb).

The node ordering schemas are:

* Natural: the order of how the nodes are loaded in the graph
* Decreasing degree: the nodes are sorted, as you have surely guessed, by decreasing node degree
* Increasing degree: the nodes are sorted, as you have surely guessed, by increasing node degree

We start by loading the graphs:

In [7]:
%%time
from tqdm.auto import tqdm
from grape.datasets.string import SaccharomycesCerevisiae, HomoSapiens, MusMusculus
from grape.datasets.kghub import KGCOVID19
from grape.datasets.networkrepository import SocFriendster

graphs = [
    graph_builder(load_nodes=False)
    for graph_builder in tqdm((
        SaccharomycesCerevisiae,
        HomoSapiens,
        MusMusculus,
        KGCOVID19,
        #SocFriendster
    ))
]

  0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 1min 1s, sys: 2.21 s, total: 1min 3s
Wall time: 21.9 s


Let's plot some of the main properties of these graphs:

In [8]:
%%time
import pandas as pd

pd.DataFrame([
    {
        "Graph": graph.get_name(),
        "Nodes": graph.get_number_of_nodes(),
        "Edges": graph.get_number_of_directed_edges(),
        "Maximum degree": graph.get_maximum_node_degree()
    }
    for graph in graphs
])

CPU times: user 10.3 ms, sys: 807 µs, total: 11.1 ms
Wall time: 1.01 ms


,Graph,Nodes,Edges,Maximum degree
0,SaccharomycesCerevisiae,6691,1988592,2729
1,HomoSapiens,19566,11938498,7507
2,MusMusculus,22048,14496358,7669
3,KGCOVID19,574232,36501154,122238


And now let's get started with benchmarking the vertex cover approaches by themselves:

In [4]:
%%time
from time import time
import numpy as np
from tqdm.auto import trange

vertex_cover_benchmarks = []

for approach in tqdm(
    ("arbitrary", "decreasing_node_degree", "increasing_node_degree"),
    leave=False
):
    for insert_only_source in tqdm((True, False), leave=False):
        for graph in tqdm(graphs, leave=False):
            start = time()
            cover = graph.get_vertex_cover(
                approach=approach,
                insert_only_source=insert_only_source
            )
            delta = time() - start
            vertex_cover_benchmarks.append({
                "Graph": graph.get_name(),
                "Approach": "{approach}{only_source}".format(
                    approach=approach,
                    only_source = " (Only source)" if insert_only_source else ""
                ),
                "Degree": max([
                    graph.get_node_degree_from_node_id(node_id)
                    for node_id, in_cover in enumerate(cover)
                    if in_cover
                ]),
                "Size": np.sum(cover),
                "Time": delta,
            })

vertex_cover_benchmarks = pd.DataFrame(vertex_cover_benchmarks)
vertex_cover_benchmarks

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

...

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/5 [00:00<?, ?it/s]

CPU times: user 2min 49s, sys: 4.52 s, total: 2min 54s
Wall time: 2min 1s


,Graph,Approach,Degree,Size,Time
0,SaccharomycesCerevisiae,arbitrary (Only source),2729,6259,0.004128
1,HomoSapiens,arbitrary (Only source),7507,19220,0.004765
2,MusMusculus,arbitrary (Only source),7669,20984,0.005488
3,KGCOVID19,arbitrary (Only source),122238,206326,0.015757
4,SocFriendster,arbitrary (Only source),5214,35466082,8.917142
5,SaccharomycesCerevisiae,arbitrary,2729,6240,0.000939
6,HomoSapiens,arbitrary,7507,19200,0.002491
7,MusMusculus,arbitrary,7669,20756,0.002816
8,KGCOVID19,arbitrary,122238,217552,0.012458
9,SocFriendster,arbitrary,5214,37992302,4.771157


Let's explore the impact of the considered vertex cover approaches on counting squares:

In [9]:
from tqdm.auto import trange
from time import time
import pandas as pd
import os
from typing import List
from grape import Graph

def experiment(graphs: List[Graph]):
    square_times = []

    for (approach, insert_only_source) in tqdm(
        (
            ("arbitrary", False),
            ("decreasing_node_degree", True),
            ("increasing_node_degree", True)
        ),
        leave=False,
        desc="Approaches"
    ):
        for graph in tqdm(graphs, leave=False, desc="Graphs"):
            global_start = time()
            number_of_squares = graph.get_number_of_squares(
                approach=approach,
                insert_only_source=insert_only_source
            )
            global_delta = time() - global_start
            per_node_start = time()
            squares_per_node = graph.get_number_of_squares_per_node(
                approach=approach,
                insert_only_source=insert_only_source
            )
            per_node_delta = time() - per_node_start
            square_times.append({
                "Graph": graph.get_name(),
                "Approach": approach,
                "Global time": global_delta,
                "Per-node time": per_node_delta,
                "Threads": os.environ["RAYON_NUM_THREADS"]
            })

    square_times = pd.DataFrame(square_times)
    return square_times

We start by using a single thread.

In [5]:
%%time
assert os.environ["RAYON_NUM_THREADS"] == "1"

square_times = experiment(graphs)
square_times

Approaches:   0%|          | 0/1 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 2h 47min 36s, sys: 9.22 s, total: 2h 47min 45s
Wall time: 2h 47min 38s


,Graph,Approach,Global time,Per-node time,Threads
0,SaccharomycesCerevisiae,arbitrary,36.004196,80.028739,1
1,HomoSapiens,arbitrary,685.894516,1532.865237,1
2,MusMusculus,arbitrary,821.426909,1821.472549,1
3,KGCOVID19,arbitrary,1615.342692,3465.803014,1


We run the same code, but now we employ six threads, half of the available number of cores:

In [5]:
%%time
assert os.environ["RAYON_NUM_THREADS"] == "6"

square_times = experiment(graphs)
square_times

Approaches:   0%|          | 0/1 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 2h 58min 34s, sys: 3.13 s, total: 2h 58min 38s
Wall time: 29min 59s


,Graph,Approach,Global time,Per-node time,Threads
0,SaccharomycesCerevisiae,arbitrary,6.146899,16.160414,6
1,HomoSapiens,arbitrary,113.805088,283.936026,6
2,MusMusculus,arbitrary,139.421086,348.075630,6
3,KGCOVID19,arbitrary,272.647228,619.538600,6


Now we run the same experiment with all of the available cores on this machine, which are 12.

In [3]:
%%time
assert os.environ["RAYON_NUM_THREADS"] == "12"

square_times = experiment(graphs)
square_times

Approaches:   0%|          | 0/1 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 3h 4min 7s, sys: 3.21 s, total: 3h 4min 10s
Wall time: 15min 22s


,Graph,Approach,Global time,Per-node time,Threads
0,SaccharomycesCerevisiae,arbitrary,3.022280,8.419647,12
1,HomoSapiens,arbitrary,57.134908,147.680663,12
2,MusMusculus,arbitrary,68.493800,181.536146,12
3,KGCOVID19,arbitrary,134.458449,321.975235,12


Next, we will evaluate how the performance change when introducing hyper-treading, using two threads per core for a total of 24.

In [10]:
%%time
assert os.environ["RAYON_NUM_THREADS"] == "24"

square_times = experiment(graphs)
square_times

Approaches:   0%|          | 0/3 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

Graphs:   0%|          | 0/4 [00:00<?, ?it/s]

CPU times: user 20h 38min 47s, sys: 2min 24s, total: 20h 41min 12s
Wall time: 1h 2min 49s


,Graph,Approach,Global time,Per-node time,Threads
0,SaccharomycesCerevisiae,arbitrary,1.828033,5.370944,24
1,HomoSapiens,arbitrary,34.278388,97.734401,24
2,MusMusculus,arbitrary,42.821282,121.405771,24
3,KGCOVID19,arbitrary,106.812338,243.399370,24
4,SaccharomycesCerevisiae,decreasing_node_degree,1.993104,5.651660,24
5,HomoSapiens,decreasing_node_degree,36.774319,96.649758,24
6,MusMusculus,decreasing_node_degree,42.384414,119.406628,24
7,KGCOVID19,decreasing_node_degree,163.761693,506.629889,24
8,SaccharomycesCerevisiae,increasing_node_degree,2.453117,7.184395,24
9,HomoSapiens,increasing_node_degree,45.256234,122.658456,24


## Conclusions

In this tutorial, we learned how to use the [GRAPE](https://github.com/AnacletoLAB/grape) library to compute the exact number of triangles in large graphs. We discussed what is a triangle, and why counting triangles can be useful. Also, we illustrated an algorithm for computing triangles using an approximated vertex cover.

I hope you now have a better grasp on computing triangles and how to use GRAPE to compute them for your projects. Do feel free to reach out with any questions or feedback, as I always look for ways to improve this tutorial.

[And remember to ⭐ GRAPE!](https://github.com/AnacletoLAB/grape)